# Thu thập dữ liệu xe ô tô

Notebook này sử dụng các scraper đã xây dựng trong thư mục `src/data` để thu thập dữ liệu xe ô tô từ các trang web.

In [32]:
import sys
import os
import pandas as pd
import time
from datetime import datetime
# Tắt logging để tránh lỗi Unicode
import logging
logging.getLogger().setLevel(logging.ERROR)  # Chỉ hiển thị lỗi nghiêm trọng

# Đường dẫn tới thư mục gốc dự án
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    
print(f"Project root: {project_root}")

Project root: c:\Users\admin\Documents\GitHub\Used_cars_prediction


In [33]:
# Tạo thư mục data/raw nếu chưa tồn tại
raw_data_dir = os.path.join(project_root, 'data', 'raw')
if not os.path.exists(raw_data_dir):
    os.makedirs(raw_data_dir)
    print(f"Đã tạo thư mục: {raw_data_dir}")
else:
    print(f"Thư mục đã tồn tại: {raw_data_dir}")

Thư mục đã tồn tại: c:\Users\admin\Documents\GitHub\Used_cars_prediction\data\raw


In [34]:
# Import scraper
# Kiểm tra nếu file scraper.py tồn tại trong thư mục hiện tại
if os.path.exists('scraper.py'):
    from scraper import BonBanhScraper
    print("Đã import BonBanhScraper từ thư mục hiện tại")
else:
    # Thử import từ thư mục src/data
    try:
        from src.data.scraper import BonBanhScraper
        print("Đã import BonBanhScraper từ src.data.scraper")
    except ImportError:
        # Nếu import từ module không thành công, import trực tiếp từ file
        import importlib.util
        
        # Đường dẫn đến file scraper.py
        scraper_path = os.path.join(project_root, 'src', 'data', 'scraper.py')
        
        if os.path.exists(scraper_path):
            spec = importlib.util.spec_from_file_location("scraper", scraper_path)
            scraper_module = importlib.util.module_from_spec(spec)
            sys.modules["scraper"] = scraper_module
            spec.loader.exec_module(scraper_module)
            BonBanhScraper = scraper_module.BonBanhScraper
            print("Đã import BonBanhScraper từ file trong src/data")
        else:
            print(f"Không tìm thấy file scraper.py tại: {scraper_path}")
            print("Vui lòng cung cấp đường dẫn đúng tới file scraper.py")

Đã import BonBanhScraper từ src.data.scraper


In [35]:
# Cấu hình đường dẫn đầu ra
output_path = os.path.join(project_root, 'data', 'raw', 'bonbanh_data.csv')
print(f"Dữ liệu sẽ được lưu vào: {output_path}")

# Khởi tạo scraper
scraper = BonBanhScraper(base_url="https://bonbanh.com", output_path=output_path)

Dữ liệu sẽ được lưu vào: c:\Users\admin\Documents\GitHub\Used_cars_prediction\data\raw\bonbanh_data.csv


In [36]:
# Nhập các tham số thu thập dữ liệu
max_pages = 10  # Số trang tối đa để thu thập (có thể thay đổi)
max_cars = 1000000  # Số lượng xe tối đa (có thể thay đổi hoặc để None nếu muốn lấy tất cả)

print(f"Chuẩn bị thu thập dữ liệu từ tối đa {max_pages} trang, với tối đa {max_cars if max_cars else 'tất cả'} xe")

Chuẩn bị thu thập dữ liệu từ tối đa 10 trang, với tối đa 1000000 xe


In [37]:
# Chạy scraper để thu thập dữ liệu
start_time = time.time()

print(f"Bắt đầu thu thập dữ liệu lúc {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Thu thập danh sách tin đăng
print("\n1. Thu thập danh sách tin đăng...")
listings = scraper.scrape_listings(max_pages=max_pages)
print(f"Đã thu thập được {len(listings)} tin đăng")

# Thu thập thông tin chi tiết
print("\n2. Thu thập thông tin chi tiết cho các xe...")
car_details = scraper.scrape_car_details(listings, max_cars=max_cars)
print(f"Đã thu thập thông tin chi tiết cho {len(car_details)} xe")

# Lưu dữ liệu vào CSV
print("\n3. Lưu dữ liệu vào file CSV...")
df = scraper.save_to_csv(car_details, output_path)
print(f"Đã lưu {len(car_details)} mẫu dữ liệu vào {output_path}")

# Tính thời gian thực hiện
elapsed_time = time.time() - start_time
print(f"\nTổng thời gian thu thập: {elapsed_time:.2f} giây")

Bắt đầu thu thập dữ liệu lúc 2025-04-12 18:05:57

1. Thu thập danh sách tin đăng...
Đã thu thập được 200 tin đăng

2. Thu thập thông tin chi tiết cho các xe...


--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 1423, in getresponse
    response.begin()
  File "c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^

KeyboardInterrupt: 

In [ ]:
# Hiển thị thông tin về dữ liệu đã thu thập
print(f"Đã thu thập {len(df)} mẫu dữ liệu với {len(df.columns)} thuộc tính")
print(f"Các cột trong dữ liệu: {df.columns.tolist()}")

# Hiển thị 5 mẫu đầu tiên
print("\n5 mẫu đầu tiên:")
df.head(5)